In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
data = pd.read_csv('housing_clean.csv')
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,3
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,3
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,3
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,3
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,3


In [5]:
label = 'median_house_value'

In [9]:
for feature in data.columns:
    if feature == label : continue

    data[feature] = (data[feature] - data[feature].min())/(data[feature].max() - data[feature].min())

In [11]:
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,0.211155,0.567481,0.784314,0.022331,0.019863,0.008941,0.020556,0.539668,452600.0,0.75
1,0.212151,0.565356,0.392157,0.180503,0.171477,0.067210,0.186976,0.538027,358500.0,0.75
2,0.210159,0.564293,1.000000,0.037260,0.029330,0.013818,0.028943,0.466028,352100.0,0.75
3,0.209163,0.564293,1.000000,0.032352,0.036313,0.015555,0.035849,0.354699,341300.0,0.75
4,0.209163,0.564293,1.000000,0.041330,0.043296,0.015752,0.042427,0.230776,342200.0,0.75


In [37]:
train_data = data[:int(0.8*len(data))]
test_data = data[int(0.8*len(data)):]
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [59]:
train_data_X = np.asarray(train_data.drop(label,axis=1))
train_data_Y = np.asarray(train_data[label])
test_data_X = np.asarray(test_data.drop(label,axis=1))
test_data_Y = np.asarray(test_data[label])

In [61]:
train_data_X.shape

(16346, 9)

In [63]:
class Node():
    def __init__(self, feature_idx=None, threshold=None, left_branch=None, right_branch=None):
        self.feature_idx = feature_idx        
        self.threshold = threshold     
        self.left_branch = left_branch    
        self.right_branch = right_branch 

In [65]:
def isClassification(dtype):
    return dtype in ['int32','int64']

def getSplit(column, threshold):
    left_ids = np.where(column <= threshold)[0]
    right_ids = np.where(column > threshold)[0]
    return left_ids, right_ids

def getImpurity(y):
    if isClassification(y.dtype):
        classes, counts = np.unique(y)
        return 1-np.sum(counts**2)/(len(y)**2)
    else:
        return np.var(y)

def getWeightedImpurity(y, left_ids, right_ids):
    total_len = len(left_ids) + len(right_ids)
    return (len(left_ids)*getImpurity(y[left_ids]) + len(right_ids)*getImpurity(y[right_ids]))/total_len

def getBestSplit(X,y):
    best_left_ids = best_right_ids = best_feature_idx = best_threshold = None
    best_impurity = float("inf") # Gets the largest float value possible
    for feature_idx in range(X.shape[1]):
        thresholds = np.unique(X[:,feature_idx])
        for threshold in thresholds:
            left_ids, right_ids = getSplit(X[:,feature_idx],threshold)
            
            if len(left_ids)*len(right_ids) == 0:
                continue
            
            weighted_impurity = getWeightedImpurity(y, left_ids, right_ids)

            if best_impurity > weighted_impurity:
                best_impurity = weighted_impurity
                best_left_ids = left_ids
                best_right_ids = right_ids
                best_feature_idx = feature_idx
                best_threshold = threshold
                
    return best_feature_idx, best_threshold, best_left_ids, best_right_ids

def mode(vals):
    unique_vals, counts = np.unique(vals,return_counts=True)
    return unique_vals[np.argmax(counts)]

def getDecisionTree(X, y, depth=0, max_depth=None, min_split=2):
    
    if ((max_depth is not None and depth >= max_depth) or getImpurity(y) == 0 or len(y) <= min_split):
        if isClassification(y.dtype): return mode(y)
        else: return np.mean(y)
    
    feature_idx, threshold, left_ids, right_ids = getBestSplit(X,y)

    if feature_idx is None:
        if isClassification(y.dtype): return mode(y)
        else: return np.mean(y)

    left_subtree = getDecisionTree(X[left_ids],y[left_ids], depth + 1, max_depth, min_split)
    right_subtree = getDecisionTree(X[right_ids],y[right_ids], depth + 1, max_depth, min_split)
    
    return Node(feature_idx, threshold, left_subtree, right_subtree)
    

In [160]:
def getPrediction(tree, x):
    if not isinstance(tree, Node):
        return tree

    feature_idx = tree.feature_idx
    threshold = tree.threshold
    if x[feature_idx] <= threshold:
        return getPrediction(tree.left_branch, x)
    else:
        return getPrediction(tree.right_branch, x)

def getPredictionAll(tree,X):
    predictions = []
    for i in range(len(X)):
        predictions.append(getPrediction(tree, X[i]))

    return np.asarray(predictions)

In [267]:
def getBootstrappedData(X, y, size=100):
    chosen_indices = np.random.randint(0, len(X), size)  # Faster than np.choice
    unchosen_indices = np.setdiff1d(np.arange(len(X)), np.unique(chosen_indices), assume_unique=True)
    return X[chosen_indices], y[chosen_indices], unchosen_indices

def fitRandomForest(X, y, size = 100, number_of_trees = 300, max_depth=7, min_samples_split=2, task="classification"):
    print(f"Fitting {number_of_trees} trees for random forest...") 
    
    trees = []

    # 2D list that stores the trees that are not using data-point i into the list at index i
    unchosen_indices_list = [[] for _ in range(len(X))]
    for tree_idx in range(number_of_trees):
        
        # Reporting progress
        if ((tree_idx+1)%(number_of_trees/20) == 0):
            print(100*(tree_idx+1)/(number_of_trees), "% completed")

        # Get the bootstrapped data
        tree_dataX, tree_dataY, unchosen_indices = getBootstrappedData(X, y, size)

        # Append the tree_idx into all the lists corresponding to unused data-point index 
        for idx in unchosen_indices:
            unchosen_indices_list[idx].append(tree_idx)
            
        trees.append(getDecisionTree(tree_dataX,tree_dataY, max_depth=max_depth, min_split = min_samples_split))
        
    return trees, unchosen_indices_list


def predictRandomForest(X,trees,task = "classification"):
    # Stores the predictions of all the trees
    predictions_all = []
    for tree in trees:
        predictions_all.append(getPredictionAll(tree,X))

    predictions_all = np.asarray(predictions_all)

    # If categorical, get the mode as the predictions else get the mean
    if task == "classification":
        predictions = []
        for i in range(predictions_all.shape[1]):
            classes, counts = np.unique(predictions_all[:,i], return_counts = True)
            predictions.append(classes[np.argmax(counts)])
    else:
        predictions = np.mean(predictions_all,axis=0)
    return np.asarray(predictions)

In [269]:
from joblib import Parallel, delayed
import time

def fitRandomForest(X, y, size=100, number_of_trees=300, max_depth=7, min_samples_split=2, task="classification"):
    print(f"Fitting {number_of_trees} trees for random forest...") 

    def train_single_tree(_):
        tree_dataX, tree_dataY, unchosen_indices = getBootstrappedData(X, y, size)
        tree = getDecisionTree(tree_dataX, tree_dataY, max_depth=max_depth, min_split=min_samples_split)
        return tree, unchosen_indices

    start_time = time.time()  # Track start time

    # Run training in parallel with progress tracking
    results = Parallel(n_jobs=-1, verbose=0)(
        delayed(train_single_tree)(_ if (i % 10 != 0) else print(f"Trained {i} trees | elapsed: {time.time() - start_time:.1f}s")) 
        for i, _ in enumerate(range(number_of_trees), 1)
    )

    print(f"[Parallel(n_jobs=-1)]: Done {number_of_trees} out of {number_of_trees} | elapsed: {time.time() - start_time:.1f}s finished")

    # Extract trees and unchosen indices from results
    trees, unchosen_indices_all = zip(*results)

    # Convert unchosen indices into a proper list structure
    unchosen_indices_list = [[] for _ in range(len(X))]
    for tree_idx, unchosen_indices in enumerate(unchosen_indices_all):
        for idx in unchosen_indices:
            unchosen_indices_list[idx].append(tree_idx)

    return list(trees), unchosen_indices_list


In [271]:
def getAccuracy(X, y, trees, task="classification"):
    prediction = predictRandomForest(X, trees, task)
    if task=="classification":
        accuracy = np.sum(prediction == y)/len(y)  # Simple accuracy for classification
    else:
        accuracy = np.mean(np.abs((y-prediction)))  # MAE for regression
    return accuracy

def getOOBError(X_train, y_train, trees, unchosen_indices_list, task="classification"):
    oob_error = 0
    for idx in range(len(unchosen_indices_list)):
        oob_trees = []
        for tree_idx in unchosen_indices_list[idx]:
            oob_trees.append(trees[tree_idx])
    
        prediction = predictRandomForest(X_train[idx:idx+1], oob_trees,task)
        if task=="classification":
            oob_error += np.sum(prediction != y_train[idx:idx+1])  # For classification
        else:
            oob_error += np.abs((prediction[0] - y_train[idx]))  # For regression
    return oob_error/len(unchosen_indices_list)

In [279]:
trees_reg, unchosen_indices_list_reg = fitRandomForest(train_data_X, train_data_Y, size = 3000, number_of_trees = 100, 
                                                       min_samples_split = 2, max_depth = None, task = "regression")

Fitting 100 trees for random forest...
Trained 10 trees | elapsed: 0.0s
Trained 20 trees | elapsed: 6.9s
Trained 30 trees | elapsed: 13.9s
Trained 40 trees | elapsed: 21.4s
Trained 50 trees | elapsed: 38.9s
Trained 60 trees | elapsed: 47.3s
Trained 70 trees | elapsed: 56.2s
Trained 80 trees | elapsed: 65.3s
Trained 90 trees | elapsed: 84.9s
Trained 100 trees | elapsed: 94.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 114.5s finished


In [280]:
print(f"MAE : {getAccuracy(test_data_X,test_data_Y,trees_reg,task="regression")}")
print(f"OOB Error : {getOOBError(train_data_X, train_data_Y, trees_reg,unchosen_indices_list_reg,task="regression")}")

MAE : 47459.78615243455
OOB Error : 33214.20186042194
